# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [5]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.multioutput import MultiOutputClassifier
from nltk.tokenize import word_tokenize
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

#nltk.download(['punkt', 'stopwords', 'wordnet', 'omw-1.4'])

In [6]:
# load data from database

df = pd.read_sql_table('messages', 'sqlite:///messages.db')
X = df['message'].values
classes_names = ['related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']
Y = df[classes_names].values

In [7]:
# Split data in train and test
xtrain, xtest, ytrain, ytest=train_test_split(X, Y, train_size=0.80, random_state=0)
print('80% of data for train = ', len(xtrain), ' messages')


80% of data for train =  20972  messages


In [8]:
X[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

# Manual clasifier fit

In [9]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [10]:
def tokenize(text):
    # Todo: normalize case and remove punctuation
    text = text.lower()
    text = re.sub(r"[^a-z0-9]", " ", text)
    # tokenize text
    tokens = word_tokenize(text)
    # lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stopwords.words('english')]
    return tokens

In [12]:
# test the tokenize function
print(tokenize(xtrain[0]))

['rue', 'dessalines', 'petit', 'goaves', 'necessary']


In [13]:
# Todo: initialize count vectorizer object and pass the tokenize function to the `tokenizer` parameter
vect = CountVectorizer(tokenizer=tokenize)

In [14]:
# Todo: get counts of each token (word) in text data (corpus) using the fit_transform method
xtrain_V = vect.fit_transform(xtrain)

In [15]:
# convert sparse matrix to numpy array to view the counts of each token (word)
# each row is one line in the text (corpus) and the number is the count of a token
#xtrain_V.toarray()

In [16]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 36)
#pd.set_option('display.width', 1000)
#df = pd.DataFrame(xtrain_V.toarray())
#df

In [17]:
# Todo: initialize tf-idf transformer object. Set smooth_idf parameter to false.
transformer = TfidfTransformer(smooth_idf = False)

In [18]:
# Todo: use counts from count vectorizer results to compute tf-idf values using the fit_transform method
xtrain_TD = transformer.fit_transform(xtrain_V)    

In [19]:
# convert sparse matrix to numpy array to view
# you can see that the counts are normalized
#xtrain_TD.toarray()

In [20]:
#from sklearn.linear_model import LogisticRegression
#lr = LogisticRegression()
#clf = MultiOutputClassifier(estimator=lr)


#from sklearn.svm import SVC
#svc = SVC(gamma="scale")
#clf = MultiOutputClassifier(estimator=svc)

from sklearn.ensemble import RandomForestClassifier
#clf = RandomForestClassifier()

rf = RandomForestClassifier()
clf = MultiOutputClassifier(estimator=rf)


In [21]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns',50)
df = pd.DataFrame(Y)
df.shape
#df

(26216, 36)

In [22]:
xtrain_TD.shape


(20972, 28165)

In [23]:
ytrain.shape

(20972, 36)

In [24]:
clf.fit(xtrain_TD, ytrain)

KeyboardInterrupt: 

In [ ]:
xtest_V = vect.transform(xtest)

In [ ]:
xtest_TD = transformer.transform(xtest_V)  

In [ ]:
ypred = clf.predict(xtest_TD)

In [ ]:
# display results
accuracy = clf.score(xtest_TD, ytest)
print (accuracy)

### 2. Write a tokenization function to process your text data

In [185]:
def tokenize_old(text):
    '''
    Fuction: tokenize the message
    Input: text to tokenize
    Output: list of tokens
    '''
    lemmatizer = WordNetLemmatizer()
    #text = X[0]
    text = text.lower() # Convert all letert to low.
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)  # Anything that isn't A through Z or 0 through 9 will be replaced by a space
    tokens = word_tokenize(text)
    tokens_wo_stopwords = [token for token in tokens if token not in stopwords.words("english")]
    #stemmed = [PorterStemmer().stem(w) for w in tokens_wo_stopwords]
    stemmed = [lemmatizer.lemmatize(w) for w in tokens_wo_stopwords]
    return stemmed

In [ ]:
def tokenize(text):
    text = text.lower() # Convert all leters to low case.
    text = re.sub(r"[^a-z0-9]", " ", text) # Anything that isn't a through z or 0 through 9 will be replaced by a space
    tokens = word_tokenize(text) # tokenize text
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stopwords.words('english')] # lemmatize and remove stop words
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [27]:
pipeline = Pipeline([
    ('vectorize', CountVectorizer(tokenizer=tokenize)),
    ('tdidf_transformation', TfidfTransformer(smooth_idf = False)),
    ('clasifier', MultiOutputClassifier(estimator=RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [38]:
# Split data in train and test
xtrain, xtest, ytrain, ytest=train_test_split(X, Y, train_size=0.9, random_state=0)
print('90% of data for train = ', len(xtrain), ' messages')

90% of data for train =  23594  messages


In [29]:
# train classifier
pipeline.fit(xtrain, ytrain)

# predict on test data
ypred = pipeline.predict(xtest)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [31]:
# display results
#accuracy = clf.score(xtest_TD, ytest)
#print (accuracy)

In [32]:
df_ypred = pd.DataFrame(ypred, columns= classes_names)
df_ytest= pd.DataFrame(ytest, columns= classes_names)
for column in list(df_ypred):
    print (column)
    print (classification_report(df_ytest[column].tolist(), df_ypred[column].tolist()))

related
              precision    recall  f1-score   support

           0       0.66      0.29      0.40      5495
           1       0.81      0.95      0.88     17935
           2       0.83      0.03      0.06       165

    accuracy                           0.79     23595
   macro avg       0.77      0.42      0.44     23595
weighted avg       0.77      0.79      0.76     23595

request
              precision    recall  f1-score   support

           0       0.89      0.98      0.93     19568
           1       0.84      0.41      0.55      4027

    accuracy                           0.89     23595
   macro avg       0.86      0.69      0.74     23595
weighted avg       0.88      0.89      0.87     23595

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23490
           1       0.00      0.00      0.00       105

    accuracy                           1.00     23595
   macro avg       0.50      0.50      0.50     2359

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23595

    accuracy                           1.00     23595
   macro avg       1.00      1.00      1.00     23595
weighted avg       1.00      1.00      1.00     23595

water
              precision    recall  f1-score   support

           0       0.95      1.00      0.97     22088
           1       0.90      0.20      0.33      1507

    accuracy                           0.95     23595
   macro avg       0.92      0.60      0.65     23595
weighted avg       0.95      0.95      0.93     23595

food
              precision    recall  f1-score   support

           0       0.94      0.99      0.96     20967
           1       0.84      0.50      0.63      2628

    accuracy                           0.93     23595
   macro avg       0.89      0.75      0.80     23595
weighted avg       0.93      0.93      0.93     23595

shelter
              precision    recall  f1-score   support


### 6. Improve your model
Use grid search to find better parameters. 

In [33]:
pipeline.get_params()

{'memory': None,
 'steps': [('vectorize',
   CountVectorizer(tokenizer=<function tokenize at 0x0000018F0A5BD820>)),
  ('tdidf_transformation', TfidfTransformer(smooth_idf=False)),
  ('clasifier', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vectorize': CountVectorizer(tokenizer=<function tokenize at 0x0000018F0A5BD820>),
 'tdidf_transformation': TfidfTransformer(smooth_idf=False),
 'clasifier': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vectorize__analyzer': 'word',
 'vectorize__binary': False,
 'vectorize__decode_error': 'strict',
 'vectorize__dtype': numpy.int64,
 'vectorize__encoding': 'utf-8',
 'vectorize__input': 'content',
 'vectorize__lowercase': True,
 'vectorize__max_df': 1.0,
 'vectorize__max_features': None,
 'vectorize__min_df': 1,
 'vectorize__ngram_range': (1, 1),
 'vectorize__preprocessor': None,
 'vectorize__stop_words': None,
 'vectorize__strip_accents': None,
 'vectorize__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vec

In [39]:
#  parameters for grid search
parameters = {
    'clasifier__estimator__criterion': ['gini', 'entropy', 'log_loss'],
    'clasifier__estimator__n_estimators': [10, 50, 100, 200]
}

#parameters = {
#    'clasifier__estimator__criterion': ['gini', 'entropy'],
#    'clasifier__estimator__n_estimators': [100]
#}

model_grid = GridSearchCV(pipeline, param_grid=parameters)

In [40]:
model_grid.fit(xtrain,ytrain)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\multioutput.py", line 351, in fit
    super().fit(X, Y, sample_weight, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\multioutput.py", line 172, in fit
    self.estimators_ = Parallel(n_jobs=self.n_jobs)(
  File "C:\ProgramData\Anaconda3\lib\site-packages\joblib\parallel.py", line 1048, in __call__
    if self.dispatch_one_batch(iterator):


GridSearchCV(estimator=Pipeline(steps=[('vectorize',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000018F0A5BD820>)),
                                       ('tdidf_transformation',
                                        TfidfTransformer(smooth_idf=False)),
                                       ('clasifier',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clasifier__estimator__criterion': ['gini', 'entropy',
                                                             'log_loss'],
                         'clasifier__estimator__n_estimators': [10, 50, 100,
                                                                200]})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [41]:
# predict on test data
ypred = model_grid.predict(xtest)

print("\nBest Parameters:", model_grid.best_params_)

df_ypred = pd.DataFrame(ypred, columns= classes_names)
df_ytest= pd.DataFrame(ytest, columns= classes_names)
for column in list(df_ypred):
    print (column)
    print (classification_report(df_ytest[column].tolist(), df_ypred[column].tolist()))


Best Parameters: {'clasifier__estimator__criterion': 'gini', 'clasifier__estimator__n_estimators': 200}
related
              precision    recall  f1-score   support

           0       0.69      0.44      0.54       620
           1       0.84      0.94      0.89      1985
           2       0.50      0.65      0.56        17

    accuracy                           0.82      2622
   macro avg       0.68      0.68      0.66      2622
weighted avg       0.81      0.82      0.80      2622

request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      2181
           1       0.84      0.51      0.64       441

    accuracy                           0.90      2622
   macro avg       0.87      0.75      0.79      2622
weighted avg       0.90      0.90      0.89      2622

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2613
           1       0.00      0.00      0.00         9

 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



              precision    recall  f1-score   support

           0       0.87      1.00      0.93      2275
           1       0.70      0.04      0.08       347

    accuracy                           0.87      2622
   macro avg       0.79      0.52      0.50      2622
weighted avg       0.85      0.87      0.82      2622

infrastructure_related
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      2460
           1       0.00      0.00      0.00       162

    accuracy                           0.94      2622
   macro avg       0.47      0.50      0.48      2622
weighted avg       0.88      0.94      0.91      2622

transport
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      2497
           1       0.95      0.14      0.25       125

    accuracy                           0.96      2622
   macro avg       0.95      0.57      0.61      2622
weighted avg       0.96      0.96      0

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [42]:
import joblib
joblib.dump(model_grid.best_estimator_, 'model.pkl', compress = 1)

['model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.